In [1]:
import pandas as pd

# Load the dataset
data = pd.read_csv("C:\\Users\\ASUS\\Desktop\\ML project\\train-metadata.csv")

# Display  information
print(data.info())


C:\Users\ASUS\AppData\Local\Temp\ipykernel_19020\304710855.py:4: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("C:\\Users\\ASUS\\Desktop\\ML project\\train-metadata.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401059 entries, 0 to 401058
Data columns (total 55 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   isic_id                       401059 non-null  object 
 1   target                        401059 non-null  int64  
 2   patient_id                    401059 non-null  object 
 3   age_approx                    398261 non-null  float64
 4   sex                           389542 non-null  object 
 5   anatom_site_general           395303 non-null  object 
 6   clin_size_long_diam_mm        401059 non-null  float64
 7   image_type                    401059 non-null  object 
 8   tbp_tile_type                 401059 non-null  object 
 9   tbp_lv_A                      401059 non-null  float64
 10  tbp_lv_Aext                   401059 non-null  float64
 11  tbp_lv_B                      401059 non-null  float64
 12  tbp_lv_Bext                   401059 non-nul

In [3]:
# Drop empty columns
data = data.dropna(axis=1, how='all')

# Fill missing values in numerical columns with mean or median
for col in data.select_dtypes(include=['float64', 'int64']).columns:
    data[col].fillna(data[col].median(), inplace=True)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_19020\3657371128.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].median(), inplace=True)


In [5]:
from sklearn.preprocessing import LabelEncoder

# Identify non-numeric columns
non_numeric_columns = data.select_dtypes(include=['object']).columns

# Apply label encoding to each non-numeric column
label_encoder = LabelEncoder()
for col in non_numeric_columns:
    data[col] = label_encoder.fit_transform(data[col].astype(str))


In [7]:
# Sample 50,000 rows
sample_data = data.sample(50000, random_state=42)

# Separate features and target
X_sample = sample_data.drop(columns=['target'])
y_sample = sample_data['target']


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.2, random_state=42)

# Train logistic regression
logreg_model = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')
logreg_model.fit(X_train, y_train)

# Predict on the test set
y_pred = logreg_model.predict(X_test)

# Evaluate the model
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9987
           1       0.92      0.85      0.88        13

    accuracy                           1.00     10000
   macro avg       0.96      0.92      0.94     10000
weighted avg       1.00      1.00      1.00     10000

Confusion Matrix:
[[9986    1]
 [   2   11]]


C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
from imblearn.over_sampling import SMOTE

# Apply SMOTE to balance the training set
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Train logistic regression on the resampled dataset
logreg_model.fit(X_train_smote, y_train_smote)

# Predict on the test set
y_pred = logreg_model.predict(X_test)

# Evaluate again
print("Classification Report after SMOTE:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix after SMOTE:")
print(confusion_matrix(y_test, y_pred))


Classification Report after SMOTE:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9987
           1       1.00      0.85      0.92        13

    accuracy                           1.00     10000
   macro avg       1.00      0.92      0.96     10000
weighted avg       1.00      1.00      1.00     10000

Confusion Matrix after SMOTE:
[[9987    0]
 [   2   11]]


C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
import pandas as pd
# Get feature importance
feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Coefficient': logreg_model.coef_[0]
}).sort_values(by='Coefficient', ascending=False)

print(feature_importance.head(10))  # Top 10 features


               Feature  Coefficient
45           iddx_full     3.630441
2           age_approx     0.564434
8             tbp_lv_A     0.272610
46              iddx_1     0.199815
9          tbp_lv_Aext     0.182206
25  tbp_lv_deltaLBnorm     0.152760
24      tbp_lv_deltaLB     0.126410
35         tbp_lv_stdL     0.116003
21       tbp_lv_deltaA     0.090403
41            tbp_lv_z     0.085535


In [19]:
# Set a threshold for coefficient importance
threshold = 0.1

# Filter features with coefficients above the threshold
important_features = feature_importance[feature_importance['Coefficient'].abs() > threshold]

# Select only these features in the dataset
X_train_filtered = X_train[important_features['Feature']]
X_test_filtered = X_test[important_features['Feature']]


In [21]:
# Train logistic regression with filtered features
logreg_model = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')
logreg_model.fit(X_train_filtered, y_train)

# Predict and evaluate
y_pred_filtered = logreg_model.predict(X_test_filtered)
print("Classification Report with Filtered Features:")
print(classification_report(y_test, y_pred_filtered))
print("Confusion Matrix with Filtered Features:")
print(confusion_matrix(y_test, y_pred_filtered))


Classification Report with Filtered Features:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9987
           1       0.93      1.00      0.96        13

    accuracy                           1.00     10000
   macro avg       0.96      1.00      0.98     10000
weighted avg       1.00      1.00      1.00     10000

Confusion Matrix with Filtered Features:
[[9986    1]
 [   0   13]]


In [27]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation
cv_scores = cross_val_score(logreg_model, X_sample, y_sample, cv=5, scoring='f1_macro')
print(f"Cross-Validation F1 Scores: {cv_scores}")
print(f"Mean F1 Score: {cv_scores.mean():.2f}")


C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 

Cross-Validation F1 Scores: [0.97056321 0.94994995 0.93470752 0.89987484 0.93470752]
Mean F1 Score: 0.94


C:\Users\ASUS\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
